In [1]:
# Cell 1: Imports
import requests
from sgp4.api import Satrec
from datetime import datetime, timedelta, timezone
import numpy as np
import pandas as pd

In [8]:
# Cell 2: Fetch TLE Data from Celestrak
# Function to fetch TLEs for a specific set (e.g., 'stations' for ISS)
def fetch_tles(set_name='stations'):
    url = f"https://celestrak.org/NORAD/elements/{set_name}.txt"
    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError(f"Failed to fetch TLE from {url}")
    lines = response.text.splitlines()
    
    tles = []
    i = 0
    while i < len(lines) - 2:
        name = lines[i].strip()
        line1 = lines[i+1].strip()
        line2 = lines[i+2].strip()
        if line1.startswith('1 ') and line2.startswith('2 '):
            tles.append({'name': name, 'line1': line1, 'line2': line2})
            i += 3
        else:
            i += 1
    return tles

# Example: Fetch TLEs for ISS (stations set)
tles = fetch_tles('stations')
print(f"Fetched {len(tles)} satellites. Example: {tles[0]}")

Fetched 13 satellites. Example: {'name': 'ISS (ZARYA)', 'line1': '1 25544U 98067A   25248.13333647  .00010878  00000+0  19750-3 0  9999', 'line2': '2 25544  51.6332 270.8228 0004213 301.3502  58.7074 15.50112942527561'}


In [ ]:
# Cell 3: Propagate Positions Using SGP4
# Function to generate positions for one satellite
def generate_positions(tle, start_time, duration_minutes=60, step_seconds=60):
    satellite = Satrec.twoline2rv(tle['line1'], tle['line2'])
    
    positions = []
    for minute in range(0, duration_minutes, step_seconds // 60):
        t = start_time + timedelta(minutes=minute)
        jd = t.toordinal() + (t - datetime(t.year, t.month, t.day)).total_seconds() / 86400.0
        fr = jd % 1
        jd = int(jd)
        
        e, r, v = satellite.sgp4(jd, fr)  # r = position in km (ECI), v = velocity in km/s
        if e == 0:  # No error
            positions.append({
                'name': tle['name'],
                'time': t.isoformat(),
                'x_km': r[0],
                'y_km': r[1],
                'z_km': r[2],
                'vx_kms': v[0],
                'vy_kms': v[1],
                'vz_kms': v[2]
            })
    
    return pd.DataFrame(positions)

# Example: Propagate for first satellite (e.g., ISS)
start_time = datetime.now(timezone.utc)  # Use current UTC time
df_positions = generate_positions(tles[0], start_time, duration_minutes=60, step_seconds=60)
print(df_positions.head())

Fetched 13 satellites. Example: {'name': 'ISS (ZARYA)', 'line1': '1 25544U 98067A   25248.13333647  .00010878  00000+0  19750-3 0  9999', 'line2': '2 25544  51.6332 270.8228 0004213 301.3502  58.7074 15.50112942527561'}
